In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 1.5 MB/s 


In [ ]:
cd /content/drive/MyDrive/knights_challenege

/content/drive/MyDrive/knights_challenege


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
import numpy as np
import os
import numpy as np
import skimage
#import skimage.io as io
import skimage.transform as transform
import torch

import numpy as np
import pandas as pd
import torch
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2
import matplotlib.pyplot as plt
import skimage 

#import logging
#import traceback
from typing import Optional, Tuple


def kits_normalization(input_image: np.ndarray):
    # first, clip to [-62, 310] (corresponds to 0.5 and 99.5 percentile in the foreground regions)
    # then, subtract 104.9 and divide by 75.3 (corresponds to mean and std in the foreground regions, respectively)
    clip_min = -62
    clip_max = 301
    mean_val = 104.0
    std_val = 75.3
    input_image = np.minimum(np.maximum(input_image, clip_min), clip_max)
    input_image -= mean_val
    input_image /= std_val
    return input_image

def normalize_to_range(input_image: np.ndarray, range: Tuple = (0.0, 1.0)):
    """
    Scales tensor to range
    @param input_image: image of shape (H x W x C)
    @param range:       bounds for normalization
    @return:            normalized image
    """
    max_val = input_image.max()
    min_val = input_image.min()
    if min_val == max_val == 0:
        return input_image
    input_image = input_image - min_val
    input_image = input_image / (max_val - min_val)
    input_image = input_image * (range[1] - range[0])
    input_image = input_image + range[0]
    return input_image

from typing import Iterable

def center_crop(np_image: np.ndarray,
                new_shape: Iterable[int],
                outside_val: float = 0
                ) -> np.ndarray:
    output_image = np.full(new_shape, outside_val, np_image.dtype)

    slices = tuple()
    offsets = tuple()
    for it, sh in enumerate(new_shape):
        size = sh // 2
        if it == 0:
            center = np_image.shape[it] - size
        else:
            center = (np_image.shape[it] // 2)
        start = center - size
        stop = center + size + (sh % 2)

        # computing what area of the original image will be in the cropped output
        slce = slice(max(0, start), min(np_image.shape[it], stop))
        slices += (slce,)

        # computing offset to pad if the crop is partly outside of the scan
        offset = slice(-min(0, start), 2 * size - max(0, (start + 2 * size) - np_image.shape[it]))
        offsets += (offset,)

    output_image[offsets] = np_image[slices]

    return output_image


def pad_image(image: np.ndarray, outer_height: int, outer_width: int, pad_value: Tuple):
    """
    Pastes input image in the middle of a larger one
    @param image:        image of shape (H x W x C)
    @param outer_height: final outer height
    @param outer_width:  final outer width
    @param pad_value:    value for padding around inner image
    @return:             padded image
    """
    inner_height, inner_width = image.shape[0], image.shape[1]
    h_offset = int((outer_height - inner_height) / 2.0)
    w_offset = int((outer_width - inner_width) / 2.0)
    outer_image = np.ones((outer_height, outer_width, 3), dtype=image.dtype) * pad_value
    outer_image[h_offset:h_offset + inner_height, w_offset:w_offset + inner_width, :] = image

    return outer_image

def normlaize_clinicalf(clinical):
    clinical['preop_egfr']=clinical['preop_egfr'].fillna(77)
    clinical["bmi"] = clinical['bmi'] /50.0
    #df["bmi"] = df['bmi'] /50.0
    clinical['age']=clinical['age']/120.0
    clinical['preop_egfr']=clinical['preop_egfr']/90.0
    clinical['radiographic_size']=clinical['radiographic_size']/15.0
    return clinical
class deeeeset(Dataset):
    def __init__(self,in_out):
        self.in_out=in_out
        
        #self.new_shape=new_shape
        #self.input_data = input_data
        self.ids=self.in_out['SubjectId']
        self.label=self.in_out['task_1_label']
        
        self.feature=self.in_out.drop(['SubjectId','gender','aua_risk_group','task_1_label','task_2_label'],axis=1, inplace=True)
        #self.feature=self.in_out.drop(['SubjectId','gender','aua_risk_group','task_1_label',],axis=1, inplace=True)
        
        #self.feature=self.in_out.drop(['SubjectId','task_1_label','task_2_label'],axis=1, inplace=True)

    
            
        
    def __getitem__(self,idx):
        
        feat=self.in_out.iloc[idx]
        #feat=pd.DataFrame(feat)
        #feat=normlaize_clinicalf(feat)
        x_feature=pd.DataFrame(feat).T

        
        # norm_img=get_array_img
        
        y_output=self.label[idx]
        x_f_array=np.array(x_feature)
        x_f_array_t=torch.from_numpy(x_f_array).float()
        x_f_array_t=torch.squeeze(x_f_array_t,axis=0)
        return (x_f_array_t,y_output)

    
    
    def __len__(self):
        return(len(self.label))
    


#input_image = center_crop(input_image, new_shape=new_shape) 
    
    
filet=pd.read_csv('/content/drive/MyDrive/knights_challenege/task1fold/train_un_moralize_fold1.csv')
#data='/home/imranr/knights/data'

filev=pd.read_csv('/content/drive/MyDrive/knights_challenege/task1fold/test_un_moralize_fold1.csv')




len(filev['task_1_label']==0)
len(filev['task_1_label']==1)

#classes=['NoAT','CanAT']
c1=filet['task_1_label'].value_counts()[0]
#175

#c2=filet['task_1_label'].value_counts()[1]
#65

train_dataset=deeeeset(filet)
valid_dataset=deeeeset(filev)

print(len(train_dataset))
print(len(valid_dataset))

f,o=valid_dataset[2]

#print(i.shape)
#print(i.min())
#print(i.max())
print(o)
print(f)

# i,f, o=train_dataset[2]

# print(i.shape)
# print(i.min())
# print(i.max())
# print(o)
# print(f)

#%
from torch.utils.data import DataLoader

    
batch=4

train_dataloader = DataLoader(train_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=True)

valid_dataloader = DataLoader(valid_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=False)

import sys 
import os
import glob
import time
import random
import os
import glob
import time
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

240
60
0
tensor([59.0000, 24.2900,  0.0000,  0.0000,  1.0000,  3.4000, 77.0000,  1.0000,
         1.0000,  0.7051,  0.7051,  3.0000])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data=next(iter(train_dataloader))
img,label=data
print(img.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([4, 12])


In [ ]:
import torch
import torch.nn as nn
class LSTMmodule(nn.Module):
  def __init__(self,input_size,h1,h2,number_layers=2):
    super().__init__()
    self.input_size=input_size
    self.h1=h1
    self.h2=h2
    self.number_layers=number_layers
    self.lstm1=nn.LSTM(input_size,h1,number_layers,batch_first=True)
    self.lstm2=nn.LSTM(h1,h2,number_layers,batch_first=True)
    self.fc=nn.Linear(256,2)
    #self.fc1=nn.Linear(256,6)

  def forward(self,x):
    #c01,h01=torch.zeros(self.number_layers,x.size(0),self.h1),torch.zeros(self.number_layers,x.size(0),self.h1).to(device)
    #c02,h02=torch.zeros(self.number_layers,x.size(0),self.h2),torch.zeros(self.number_layers,x.size(0),self.h2).to(device)

    out1,(c1,h1)=self.lstm1(x)
    out2,(c2,h2)=self.lstm2(out1)
    #print(out2[:,-1,:].shape) # batch_size,hidden_out
    out=self.fc(out2[:,-1,:])
    return out
model=LSTMmodule(12,128,256,2)
#inpt=torch.randn(4,1,12) # batch_size,seq_len=7,input_size=10
#out=model(inpt)
#print(out.shape)

In [ ]:
cd /content/drive/MyDrive/knights_challenege

/content/drive/MyDrive/knights_challenege


In [ ]:
!mkdir models

In [ ]:
cd /content/drive/MyDrive/knights_challenege/models

/content/drive/MyDrive/knights_challenege/models


In [ ]:
!mkdir fold0MLasmfeatures

In [ ]:
!mkdir fold0modelGRU

In [ ]:
cd /content/drive/MyDrive/knights_challenege

/content/drive/MyDrive/knights_challenege


In [ ]:
import torch
import torch.nn as nn
class GRUmodule(nn.Module):
  def __init__(self,input_size,h1,h2,number_layers=2):
    super().__init__()
    self.input_size=input_size
    self.h1=h1
    self.h2=h2
    self.number_layers=number_layers
    self.gru1=nn.GRU(input_size,h1,self.number_layers,batch_first=True)
    self.gru2=nn.GRU(h1,h2,self.number_layers,batch_first=True)
    self.fc=nn.Linear(256,2)

  def forward(self,x):
    #h01=torch.zeros(self.number_layers,x.size(0),self.h1)
    #h1=torch.zeros(self.number_layers,x.size(0),self.h2)
    out1,_=self.gru1(x)
    out2,_=self.gru2(out1)
    #print(out2[:,-1,:].shape)
    #out=out2[:,-1,:]
    out=self.fc(out2[:,-1,:])
    return out
model=GRUmodule(12,128,256)

In [ ]:
import torch
import torch.nn as nn
class CNN_model(nn.Module):
  def __init__(self,in_chan,classes):
    super(CNN_model,self).__init__()
    # block 1
    self.c11=nn.Conv1d(in_channels=in_chan,
                       out_channels=120,kernel_size=3) # 12-3+1=10
    #self.maxpool11=nn.MaxPool1d(2) #58
    self.c21=nn.Conv1d(in_channels=120,
                       out_channels=130,kernel_size=3) #10-3+1=8
    #self.maxpool21=nn.MaxPool1d(2) #28
    # block2
    self.c12=nn.Conv1d(in_channels=130,
                       out_channels=140,kernel_size=3) #8-3+1=6
    self.maxpool12=nn.MaxPool1d(2) #6/2=3
    self.c22=nn.Conv1d(in_channels=140,
                       out_channels=64,kernel_size=1) #3-1+1=3 
    #self.maxpool13=nn.MaxPool1d(3) #15/3=102
    # linear layer
    self.fc1=nn.Linear(64*3,128)
    self.fc2=nn.Linear(128,classes)

  def forward(self,x):
    # block 1 
    #x=self.maxpool21(self.c21(self.maxpool11(self.c11(x))))
    x=self.c12(self.c21(self.c11(x)))
    #x=(self.c22(self.maxpool12(self.c12(x))))
    x=self.maxpool12(x)
    x=self.c22(x)
    x=x.view(-1,64*3)
    x=self.fc1(x)
    x=self.fc2(x)
    return x
model=CNN_model(in_chan=1,classes=2) # inp,number_layers,hidden_dim#
#inp=torch.rand(1,1,12)
#out=model(inp)
#print(out.shape)

In [ ]:
from typing import Optional, Sequence
import torch.nn as nn
class ClassifierMLP(nn.Module):
    def __init__(self, in_ch: int, num_classes: Optional[int], layers_description: Sequence[int]=(256,128), dropout_rate: float = 0.1):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Linear(in_ch, layers_description[0]))
        layer_list.append(nn.ReLU())
        if dropout_rate is not None and dropout_rate > 0:
            layer_list.append(nn.Dropout(p=dropout_rate))
        last_layer_size = layers_description[0]
        for curr_layer_size in layers_description[1:]:
            layer_list.append(nn.Linear(last_layer_size, curr_layer_size))
            layer_list.append(nn.ReLU())
            if dropout_rate is not None and dropout_rate > 0:
                layer_list.append(nn.Dropout(p=dropout_rate))
            last_layer_size = curr_layer_size
        
        if num_classes is not None:
            layer_list.append(nn.Linear(last_layer_size, num_classes))
        
        self.classifier = nn.Sequential(*layer_list)

    def forward(self, x):
        x = self.classifier(x)
        return x
  
model=ClassifierMLP(12,2)  
import torch    

In [ ]:
#### training and testing functions
#model=nn.DataParallel(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
#loss_func=nn.BCEWithLogitsLoss()
from tqdm import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.00001)

#classes=['NRG','RG']
classes=['NoAT','CanAT']
#c1=data['label'].value_counts()[0]
#c2=data['label'].value_counts()[1]
my_distribution=np.array([175,65])
class_weights = torch.from_numpy(np.divide(1, my_distribution)).float().to(device)
class_weights = class_weights / class_weights.sum()
for i, c in enumerate(classes):
  print('Weight for class %s: %f' % (c, class_weights.cpu().numpy()[i]))
loss_func = nn.CrossEntropyLoss(weight=class_weights)
#loss_func=nn.CrossEntropyLoss()
#label=label.to(torch.int64)
################################ training functions ###################
def train_fn(model,train_loader):
    model.train()
    counter=0
    training_run_loss=0.0
    train_running_correct=0.0
    for i, data in tqdm(enumerate(train_loader),total=int(len(train_dataset)/train_loader.batch_size)):
        counter+=1
        # extract dataset
        feature,label=data
        #imge=imge.float()
        #label=label.float()
        label.to(torch.int64)
        feature=feature.float()
        feature=feature.to(device)
        #imge=imge.to(device)
        label=label.to(device)
        #imge=imge.cuda()
        #label=label.cuda()
        # zero_out the gradient
        optimizer.zero_grad()
        output=model(feature)
        loss=loss_func(output,label)
        training_run_loss+=loss.item()
        _,preds=torch.max(output.data,1)
        train_running_correct+=(preds==label).sum().item()
        loss.backward()
        optimizer.step()
    ###################### state computation ###################
    train_loss=training_run_loss/len(train_loader.dataset)
    train_loss_ep.append(train_loss)
    train_accuracy=100.* train_running_correct/len(train_loader.dataset)
    train_accuracy_ep.append(train_accuracy)
    print(f"Train Loss:{train_loss:.4f}, Train Acc:{train_accuracy:0.2f}")
    return train_loss_ep,train_accuracy_ep

########################## validation function ##################
def validation_fn(model,valid_loader):
  # evluation start
    print("validation start")
    
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i,data in tqdm(enumerate(valid_loader),total=int(len(valid_loader)/valid_loader.batch_size)):
            feature,label=data
            #imge=imge.float()
            #label=label.float()
            label.to(torch.int64)
            feature=feature.float()
            feature=feature.to(device)
            #imge=imge.to(device)
            label=label.to(device)
            #imge=imge.cuda()
            #label=label.cuda()
            output=model(feature)
            loss=loss_func(output,label)
            val_running_loss+=loss.item()
            _,pred=torch.max(output.data,1)
            val_running_correct+=(pred==label).sum().item()
        val_loss=val_running_loss/len(valid_loader.dataset)
        val_loss_ep.append(val_loss)
        val_accuracy=100.* val_running_correct/(len(valid_loader.dataset))
        val_accuracy_ep.append(val_accuracy)
        print(f"Val Loss:{val_loss:0.4f}, Val_Acc:{val_accuracy:0.2f}")
        return val_loss_ep,val_accuracy_ep

Weight for class NoAT: 0.270833
Weight for class CanAT: 0.729167


In [ ]:
cd /content/drive/MyDrive/knights_challenege/

/content/drive/MyDrive/knights_challenege/models


In [ ]:
!mkdir fold0GRUNorm

In [ ]:
!mkdir fold0CNNnewNorm

In [ ]:
cd /content/drive/MyDrive/knights_challenege/

/content/drive/MyDrive/knights_challenege


In [ ]:
mkdir fold1mlptask1un

In [ ]:
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.0001)
train_loss_ep=[]
train_accuracy_ep=[]
val_loss_ep=[]
val_accuracy_ep=[]
lr = 3e-4
log = pd.DataFrame(index=[], columns=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])
early_stop=20
epochs=1000
best_acc = 0
name='fold1mlptask1un'
trigger = 0
for epoch in range(epochs):
    print('Epoch [%d/%d]' %(epoch, epochs))
    # train for one epoch
    train_loss_ep,train_accuracy_ep=train_fn(model,train_dataloader)
    train_loss_ep1=np.mean(train_loss_ep)
    train_accuracy_ep1=np.mean(train_accuracy_ep)
    #y_pred,labels=Prediciton_fn(model,valid_loader)

    val_loss_ep,val_accuracy_ep=validation_fn(model,valid_dataloader)
    val_loss_ep1=np.mean(val_loss_ep)
    val_accuracy_ep1=np.mean(val_accuracy_ep)
    
    print('loss %.4f - accu %.4f - val_loss %.4f - val_accu %.4f'%(train_loss_ep1, train_accuracy_ep1, val_loss_ep1, val_accuracy_ep1))

    tmp = pd.Series([epoch,lr,train_loss_ep1,train_accuracy_ep1,val_loss_ep1,val_accuracy_ep1], index=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('models/%s/logmlp.csv' %name, index=False)

    trigger += 1

    if val_accuracy_ep1 > best_acc:
        torch.save(model.state_dict(), 'models/%s/mlpmodelfold1.pth' %name)
        best_acc = val_accuracy_ep1
        print("=> saved best model")
        trigger = 0

    # early stopping
    if not early_stop is None:
        if trigger >= early_stop:
            print("=> early stopping")
            break

    torch.cuda.empty_cache() 

Epoch [0/1000]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 60/60 [00:00<00:00, 64.93it/s]

Train Loss:0.3286, Train Acc:54.58
validation start



15it [00:00, 27.89it/s]

Val Loss:0.1396, Val_Acc:70.00
loss 0.3286 - accu 54.5833 - val_loss 0.1396 - val_accu 70.0000
=> saved best model
Epoch [1/1000]


...

Val Loss:0.0888, Val_Acc:86.67
loss 0.1288 - accu 74.9058 - val_loss 0.0985 - val_accu 85.3458
Epoch [429/1000]



100%|██████████| 60/60 [00:01<00:00, 48.10it/s]

Train Loss:0.1102, Train Acc:79.58
validation start



15it [00:00, 18.84it/s]


Val Loss:0.0915, Val_Acc:85.00
loss 0.1287 - accu 74.9167 - val_loss 0.0985 - val_accu 85.3450
=> early stopping


In [ ]:
import torch
import torch.nn as nn
class GRUmodule(nn.Module):
  def __init__(self,input_size,h1,h2,number_layers=2):
    super().__init__()
    self.input_size=input_size
    self.h1=h1
    self.h2=h2
    self.number_layers=number_layers
    self.gru1=nn.GRU(input_size,h1,self.number_layers,batch_first=True)
    self.gru2=nn.GRU(h1,h2,self.number_layers,batch_first=True)
    self.fc=nn.Linear(256,2)

  def forward(self,x):
    #h01=torch.zeros(self.number_layers,x.size(0),self.h1)
    #h1=torch.zeros(self.number_layers,x.size(0),self.h2)
    out1,_=self.gru1(x)
    out2,_=self.gru2(out1)
    #print(out2[:,-1,:].shape)
    #out=out2[:,-1,:]
    out=self.fc(out2[:,-1,:])
    return out
model=GRUmodule(12,128,256)

In [ ]:
import torch
import torch.nn as nn
class CNN_model(nn.Module):
  def __init__(self,in_chan,classes):
    super(CNN_model,self).__init__()
    # block 1
    self.c11=nn.Conv1d(in_channels=in_chan,
                       out_channels=120,kernel_size=3) # 12-3+1=10
    #self.maxpool11=nn.MaxPool1d(2) #58
    self.c21=nn.Conv1d(in_channels=120,
                       out_channels=130,kernel_size=3) #10-3+1=8
    #self.maxpool21=nn.MaxPool1d(2) #28
    # block2
    self.c12=nn.Conv1d(in_channels=130,
                       out_channels=140,kernel_size=3) #8-3+1=6
    self.maxpool12=nn.MaxPool1d(2) #6/2=3
    self.c22=nn.Conv1d(in_channels=140,
                       out_channels=64,kernel_size=1) #3-1+1=3 
    #self.maxpool13=nn.MaxPool1d(3) #15/3=102
    # linear layer
    self.fc1=nn.Linear(64*3,128)
    self.fc2=nn.Linear(128,classes)

  def forward(self,x):
    # block 1 
    #x=self.maxpool21(self.c21(self.maxpool11(self.c11(x))))
    x=self.c12(self.c21(self.c11(x)))
    #x=(self.c22(self.maxpool12(self.c12(x))))
    x=self.maxpool12(x)
    x=self.c22(x)
    x=x.view(-1,64*3)
    x=self.fc1(x)
    x=self.fc2(x)
    return x
model=CNN_model(in_chan=1,classes=2) # inp,number_layers,hidden_dim#
inp=torch.rand(1,1,12)
out=model(inp)
print(out.shape)

torch.Size([1, 2])


In [ ]:
### 1DCNN+LSTM model
import torch
import torch.nn as nn
class CNNLSTM(nn.Module):
  def __init__(self,input_size,number_layers,hidden_units,number_classes=2):
    super().__init__()
    self.number_layers=number_layers
    self.hidden_units=hidden_units
    self.input_size=input_size
    self.c1=nn.Conv1d(in_channels=input_size,out_channels=10,kernel_size=5) # 120+1-5=116
    self.b1=nn.BatchNorm1d(10)
    self.mp1=nn.MaxPool1d(2) #116/2=58
    self.c2=nn.Conv1d(10,out_channels=20,kernel_size=3) #58+1-3=56
    self.b2=nn.BatchNorm1d(20)
    self.mp2=nn.MaxPool1d(2) #56/2=28
    # b*20*30
    self.lstm=nn.LSTM(28,self.hidden_units,self.number_layers,batch_first=True)
    self.fc=nn.Linear(self.hidden_units,number_classes)

  def forward(self,x):
    x=self.mp1(self.b1(self.c1(x)))
    x=self.mp2(self.b2(self.c2(x)))
    #print(x.shape)
    # h0=torch.zeros(self.number_layers,x.size(0),self.hidden_units)
    # c0=torch.zeros(self.number_layers,x.size(0),self.hidden_units)
    # h0.cuda()
    # c0.cuda()
    # out,(h1,c1)=self.lstm(x,(h0,c0))
    out,(h1,c1)=self.lstm(x)
    out1=self.fc(out[:,-1,:]) # batch_size*hidden_units
    #print(out1.shape)
    return out1
model=CNNLSTM(70,2,64) # inp,number_layers,hidden_dim
# inpt=torch.randn(10,64,120) # batch_sizexfeatures or input channels xseq_len in time 

In [ ]:
#### training and testing functions
model=nn.DataParallel(model)
model=model.to(device)
#loss_func=nn.BCEWithLogitsLoss()
from tqdm import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.00001)

#classes=['NRG','RG']
classes=['NoAT','CanAT']
#c1=data['label'].value_counts()[0]
#c2=data['label'].value_counts()[1]
my_distribution=np.array([175,65])
class_weights = torch.from_numpy(np.divide(1, my_distribution)).float().to(device)
class_weights = class_weights / class_weights.sum()
for i, c in enumerate(classes):
  print('Weight for class %s: %f' % (c, class_weights.cpu().numpy()[i]))
loss_func = nn.CrossEntropyLoss(weight=class_weights)
#label=label.to(torch.int64)
################################ training functions ###################
def train_fn(model,train_loader):
    model.train()
    counter=0
    training_run_loss=0.0
    train_running_correct=0.0
    for i, data in tqdm(enumerate(train_loader),total=int(len(train_dataset)/train_loader.batch_size)):
        counter+=1
        # extract dataset
        feature,label=data
        #imge=imge.float()
        #label=label.float()
        label.to(torch.int64)
        feature=feature.float()
        feature=feature.to(device)
        #imge=imge.to(device)
        label=label.to(device)
        #imge=imge.cuda()
        #label=label.cuda()
        # zero_out the gradient
        optimizer.zero_grad()
        output=model(feature)
        loss=loss_func(output,label)
        training_run_loss+=loss.item()
        _,preds=torch.max(output.data,1)
        train_running_correct+=(preds==label).sum().item()
        loss.backward()
        optimizer.step()
    ###################### state computation ###################
    train_loss=training_run_loss/len(train_loader.dataset)
    train_loss_ep.append(train_loss)
    train_accuracy=100.* train_running_correct/len(train_loader.dataset)
    train_accuracy_ep.append(train_accuracy)
    print(f"Train Loss:{train_loss:.4f}, Train Acc:{train_accuracy:0.2f}")
    return train_loss_ep,train_accuracy_ep

########################## validation function ##################
def validation_fn(model,valid_loader):
  # evluation start
    print("validation start")
    
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i,data in tqdm(enumerate(valid_loader),total=int(len(valid_loader)/valid_loader.batch_size)):
            feature,label=data
            #imge=imge.float()
            #label=label.float()
            label.to(torch.int64)
            feature=feature.float()
            feature=feature.to(device)
            #imge=imge.to(device)
            label=label.to(device)
            #imge=imge.cuda()
            #label=label.cuda()
            output=model(feature)
            loss=loss_func(output,label)
            val_running_loss+=loss.item()
            _,pred=torch.max(output.data,1)
            val_running_correct+=(pred==label).sum().item()
        val_loss=val_running_loss/len(valid_loader.dataset)
        val_loss_ep.append(val_loss)
        val_accuracy=100.* val_running_correct/(len(valid_loader.dataset))
        val_accuracy_ep.append(val_accuracy)
        print(f"Val Loss:{val_loss:0.4f}, Val_Acc:{val_accuracy:0.2f}")
        return val_loss_ep,val_accuracy_ep

In [ ]:
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.0001)
train_loss_ep=[]
train_accuracy_ep=[]
val_loss_ep=[]
val_accuracy_ep=[]
lr = 3e-4
log = pd.DataFrame(index=[], columns=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])
early_stop=20
epochs=500
best_acc = 0
name='fold4model'
trigger = 0
for epoch in range(epochs):
    print('Epoch [%d/%d]' %(epoch, epochs))
    # train for one epoch
    train_loss_ep,train_accuracy_ep=train_fn(model,train_dataloader)
    train_loss_ep1=np.mean(train_loss_ep)
    train_accuracy_ep1=np.mean(train_accuracy_ep)
    #y_pred,labels=Prediciton_fn(model,valid_loader)

    val_loss_ep,val_accuracy_ep=validation_fn(model,valid_dataloader)
    val_loss_ep1=np.mean(val_loss_ep)
    val_accuracy_ep1=np.mean(val_accuracy_ep)
    
    print('loss %.4f - accu %.4f - val_loss %.4f - val_accu %.4f'%(train_loss_ep1, train_accuracy_ep1, val_loss_ep1, val_accuracy_ep1))

    tmp = pd.Series([epoch,lr,train_loss_ep1,train_accuracy_ep1,val_loss_ep1,val_accuracy_ep1], index=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('models/%s/log.csv' %name, index=False)

    trigger += 1

    if val_accuracy_ep1 > best_acc:
        torch.save(model.state_dict(), 'models/%s/3dmodeltask1fold4.pth' %name)
        best_acc = val_accuracy_ep1
        print("=> saved best model")
        trigger = 0

    # early stopping
    if not early_stop is None:
        if trigger >= early_stop:
            print("=> early stopping")
            break

    torch.cuda.empty_cache() 